In [39]:
!pip install -q playwright nest_asyncio
!playwright install chromium
import nest_asyncio
import asyncio
import pandas as pd
from playwright.async_api import async_playwright
import re
nest_asyncio.apply()


In [40]:
_BASE_URL_='https://www.zenga.hu'
def url_creator(place:str="pest",type:str="haz",ar_min=40000000, ar_max=100000000):

    return f"{_BASE_URL_}/{place}+elado+{type}+ar-{ar_min}-{ar_max}?page="

In [41]:
url_creator("urom")

'https://www.zenga.hu/urom+elado+haz+ar-40000000-100000000?page='

In [42]:
async def scrape_nof_links(start_url="https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page=",page_num=1):

    start_url = start_url+str(page_num)
    links = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(start_url)
        await page.wait_for_timeout(8000)

        # Scroll, hogy több kártya betöltődjön
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(6000)

        div_list = await page.query_selector_all("div[data-cy='search-page-total-count']") # linkek itt vannak a html-ben
        text=await div_list[0].text_content()
        print(f"Talált linkek száma: {text}")
        number_of_links = int(re.search(r'\d+', text.replace('\u00A0', '').replace(' ', '')).group())


        await browser.close()
        return number_of_links

In [43]:
number_of_links=await scrape_nof_links()
print(f"A keresésnek megfelelő linkek száma:{number_of_links}")

Talált linkek száma:  1 859 találat 
A keresésnek megfelelő linkek száma:1859


In [44]:
async def scrape_zenga_links(start_url="https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page=",page=1):

    start_url = start_url+str(page)
    links = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(start_url)
        await page.wait_for_timeout(8000)

        # Scroll, hogy több kártya betöltődjön
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(3000)

        a_links = await page.query_selector_all("a[data-cy='advert-card-link']") # linkek itt vannak a html-ben
        print(f"Talált linkek száma: {len(a_links)}")

        for a in a_links:
            href = await a.get_attribute("href")
            if href:
                links.append(_BASE_URL_ + "/" + href)

        await browser.close()
        return links

In [47]:
def get_all_url(place:str,type="haz",l_price=40000000, h_price=100000000)->list:
    url=url_creator(place=place,type=type,ar_min=l_price,ar_max=h_price)
    print(url)
    zenga_links=[]

    no_links = asyncio.run(scrape_nof_links(start_url=url,page_num=1))
    # Futtatás

    for i in range(-(-no_links // 20)):
        print(f"Page: {i+1}")
        zenga_page = asyncio.run( scrape_zenga_links(start_url=url,page=i+1))
        zenga_links.extend(zenga_page)
    # Kiírás
    for i, link in enumerate(zenga_links, 1):
        print(f"{i}. {link}")
    return zenga_links


In [48]:
zenga_links=get_all_url("torokbalint","haz",40000000,100000000)

https://www.zenga.hu/torokbalint+elado+haz+ar-40000000-100000000?page=
Talált linkek száma:  23 találat 
Page: 1
Talált linkek száma: 20
Page: 2
Talált linkek száma: 3
1. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8603944?page=1&pos=1&cr=50
2. https://www.zenga.hu//ingatlan/elado-ikerhaz-torokbalint/7843274?page=1&pos=2&cr=5
3. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8624680?page=1&pos=3
4. https://www.zenga.hu//ingatlan/elado-hazresz-torokbalint/8481940?page=1&pos=4
5. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8642644?page=1&pos=5
6. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8506373?page=1&pos=6
7. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8649772?page=1&pos=7
8. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint-ofalu/8059934?page=1&pos=8
9. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint-ofalu/8291269?page=1&pos=9
10. https://www.zenga.hu//ingatlan/elado-csaladi-haz

In [49]:
df=pd.DataFrame(zenga_links,columns=['link'])

In [50]:
csv_path='/content/drive/MyDrive/Zenga/'

In [51]:
df.to_csv(csv_path+'links_torokbalint.csv',index=False)

In [52]:
async def scrape_ad_text(url):
    results = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()


        try:
            print(f"🔗 {url}")
            await page.goto(url, timeout=20000)
            try:
                await page.get_by_text("Összes süti engedélyezése").click()
                print("Süti elfogadva")
            except:
                print("Süti gomb nem jelent meg")


            await page.wait_for_timeout(7000)
            #await page.screenshot(path="screenshot.png")  # DEBUG
            desc_element = await page.query_selector("span[data-cy='advert-details-description']")
            desc_text = await desc_element.inner_text() if desc_element else "Nincs szöveg"
            print(f"Szöveg: {desc_text} ")

            price_element = await page.query_selector("div[data-cy='advert-details-price']")
            price_text = await price_element.inner_text() if desc_element else "Nincs ár"
            print(f"Ár: {price_text} ")


            results.append({"url": url, "desc": desc_text, "price":price_text})
        except Exception as e:
            print(f"Hiba: {str(e)}")
            results.append({"url": url, "text": f"Hiba: {str(e)}"})

        await browser.close()

    return results  #return a list


In [53]:
def scrape_zenga_list(z_list_list):
    out_list=[]
    for url in z_list_list:
        url_data = asyncio.run(scrape_ad_text(url))
        print(url_data)
        out_list.extend(url_data)
    return out_list

In [54]:
zenga_data=scrape_zenga_list(zenga_links)

🔗 https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8603944?page=1&pos=1&cr=50
Süti elfogadva
Szöveg: Kizárólag a mi irodánk kínálatában!!!

Egyedi lehetőség alakítsa ki saját ízlése szerint!
Befektetésnek is egyaránt alkalmas mivel a ház három lakrészre van osztva , ami áll két garzon lakás részből.
A nagy ház egy amerikai konyhás nappali fürdőszoba előszoba és az emeleten nagy tágas nappali és két különálló szobából.

- Kiadásra is remek lehetőség.
Az ingatlan tehermentes rendezett tulajdoni viszonyú akár gyorsan birtokba vehető.
Bővebb információért kérem hívjon bizalommal akár hétvégén is. 
Ár: 58 millió Ft 
[{'url': 'https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8603944?page=1&pos=1&cr=50', 'desc': 'Kizárólag a mi irodánk kínálatában!!!\n\nEgyedi lehetőség alakítsa ki saját ízlése szerint!\nBefektetésnek is egyaránt alkalmas mivel a ház három lakrészre van osztva , ami áll két garzon lakás részből.\nA nagy ház egy amerikai konyhás nappali fürdőszoba elősz

In [55]:
df=pd.DataFrame(zenga_data)
csv_path='/content/drive/MyDrive/Zenga/'
df.to_csv(csv_path+'zenga_data_torokbalint.csv',index=False)

In [56]:
szoveg=df.desc[0]

In [57]:
raise SystemExit("🛑 Kód megszakítva ezen a ponton.") # A következő rész a AI prompt beállításait tartalmazza

SystemExit: 🛑 Kód megszakítva ezen a ponton.

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [58]:
from openai import OpenAI
import json

In [59]:
from google.colab import userdata
key=userdata.get('OpenAi_API')

In [60]:
szoveg='''
Budafok szép panorámás részén a Rózsavölgy feletti dombon, csendes mellékutcában eladó egy bájos kis ház 520 négyzetméteres telken.

A kis ház 35 négyzetméteres, egy légtérben van kialakítva benne a nappali, melyet részben nyitott válaszfallal szeparáltak el a konyha-étkező résztől. A zuhanyzós fürdő és a WC egy helyiségben található.

A víz, az áram és az internet be van kötve a házba. Gáz és csatorna az utcában. A csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.

A telken áll egy másik kis házikó is, ami jelenleg egy 9 négyzetméteres szobaként funkcionál.

A kert szép és gondozott, sok gyümölcsfával. Nagyobb mérete miatt sok lehetőséget rejt. Kialakítható rajta több nagy magaságyás, építhető kerti medence vagy finn szauna is.

További információkért és időpont egyeztetésért kérem hívjon telefononn.
Hitellel vásárlás esetén bankfüggetlen és DÍJMENTES hiteltanácsadó közreműködését biztosítjuk, aki a teljes folyamaton végigkíséri Önt.
Az adásvétel biztonságos és gördülékeny bonyolítása érdekében tapasztalt ingatlan szakjogászaink szolgáltatását is ajánlom.
'''

In [61]:
def create_prompt(szoveg):
    prompt_message=[
            {"role": "system",
             "content": f'''
                    Te egy ingatlanhirdetés-elemző nyelvész vagy, komoly ingatlanhirdetési tapasztalattal.
                    Célod, hogy az ingatlan hirdetések tökéletesek legyenek ezért kidolgoztál egy hirdetés értékelési szempontrendszert.
                    Ezek a szempontok alapján tökéletes értékelést tudsz adni az adott hirdetés szövege alapján.
                    A szempontok:
                        1. Érthetőség
                        2. Részletesség / információtartalom
                        3. Szerkezet, logikai felépítés
                        4. Célcsoport megszólítása
                        5. Stílus és nyelvhelyesség
                        6. Előnyök kiemelése
                        7. Negatívumok őszinte kezelése
                        8. Eladásra ösztönzés'''},
            {"role": "user",
             "content": """
                            Értékeld az alábbi ingatlanhirdetés szöveget és az értékelést a scoring mezőbe helyezzed el.
                            Az alábbi szempontok alapján 1–5-ig pontozzad, és hozz létre egy összesített eredményt  1-8 közötti szempontok értékelésének átlagaként ez legyen a 9. Összesítés :
                            Majd a tudásod alapján adj egy alternatív javaslatot az eredeti szöveg javítására, hogy a lehető legjobban megfeleljen a szempontoknak.
                            Az alternatív javaslatot strukturáld, és tördeld a jobb érthetőség érdekében, de markdown formátumot ne használj!
                            Fontos, hogy a saját szempontrendszered szerint az alternatív javaslatod értékelése jobb legyen az eredeti értékelésnél!
                            A válaszod csak érvényes JSON formátumban legyen, pontosan az alábbi struktúrában:

                            {"scoring":
                                {

                                    "Érthetőség": <szám>,
                                    "Részletesség": <szám>,
                                    "Szerkezet": <szám>,
                                    "Célcsoport": <szám>,
                                    "Stílus": <szám>,
                                    "Előnyök": <szám>,
                                    "Negatívumok": <szám>,
                                    "Ösztönzés": <szám>,
                                    "Összesítés": <szám.tizedes>"
                                },
                            "proposal":"Ide kerüljön a javított szöveg javaslatod"
                            }


                    A HIRDETÉS SZÖVEGE:
                        """+f" {szoveg}"}
        ]
    return prompt_message


In [62]:
prompt_message=create_prompt(szoveg)


In [63]:
def get_response(szoveg:str):
    client = OpenAI(api_key=key)
    prompt_message=create_prompt(szoveg)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=prompt_message,
        temperature=0.5,
        max_tokens=2900
    )

    content = response.choices[0].message.content

    # a gpt-4 így adja vissza a json választ, kiszedjük a jsont a felesleges keretből
    if content.startswith("```json"):
        content = content.strip("`").lstrip("json").strip()

    #
    out_dict = json.loads(content)

    return(out_dict)

In [64]:
resp_data=get_response(prompt_message)

In [65]:
print(resp_data)

{'scoring': {'Érthetőség': 4, 'Részletesség': 4, 'Szerkezet': 3, 'Célcsoport': 3, 'Stílus': 4, 'Előnyök': 4, 'Negatívumok': 3, 'Ösztönzés': 4, 'Összesítés': 3.6}, 'proposal': 'Eladó egy bájos kis ház Budafok szép panorámás részén, a Rózsavölgy feletti dombon, csendes mellékutcában, 520 négyzetméteres telken.\n\nA ház 35 négyzetméteres, egy légterű nappalival, amelyet részben nyitott válaszfal választ el a konyha-étkezőtől. A fürdőszoba zuhanyzós, a WC-vel egy helyiségben található.\n\nA ház rendelkezik víz-, áram- és internetkapcsolattal. A gáz és a csatorna az utcában elérhető, a csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.\n\nA telken található egy másik kis házikó is, amely jelenleg 9 négyzetméteres szobaként funkcionál.\n\nA kert gondozott, sok gyümölcsfával, és nagyobb mérete miatt sok lehetőséget rejt. Kialakítható rajta magaságyás, építhető kerti medence vagy finn szauna.\n\nTovábbi információkért és időpont egyeztetésért kérem, hívjon telefonon.\n\nHit

In [66]:
resp_data["scoring"]

{'Érthetőség': 4,
 'Részletesség': 4,
 'Szerkezet': 3,
 'Célcsoport': 3,
 'Stílus': 4,
 'Előnyök': 4,
 'Negatívumok': 3,
 'Ösztönzés': 4,
 'Összesítés': 3.6}

In [67]:
resp_data["proposal"]

'Eladó egy bájos kis ház Budafok szép panorámás részén, a Rózsavölgy feletti dombon, csendes mellékutcában, 520 négyzetméteres telken.\n\nA ház 35 négyzetméteres, egy légterű nappalival, amelyet részben nyitott válaszfal választ el a konyha-étkezőtől. A fürdőszoba zuhanyzós, a WC-vel egy helyiségben található.\n\nA ház rendelkezik víz-, áram- és internetkapcsolattal. A gáz és a csatorna az utcában elérhető, a csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.\n\nA telken található egy másik kis házikó is, amely jelenleg 9 négyzetméteres szobaként funkcionál.\n\nA kert gondozott, sok gyümölcsfával, és nagyobb mérete miatt sok lehetőséget rejt. Kialakítható rajta magaságyás, építhető kerti medence vagy finn szauna.\n\nTovábbi információkért és időpont egyeztetésért kérem, hívjon telefonon.\n\nHitellel vásárlás esetén díjmentes, bankfüggetlen hiteltanácsadó segítségét biztosítjuk, aki végigkíséri Önt a teljes folyamaton.\nAz adásvétel biztonságos lebonyolítása érdekébe

Eladásra kínálunk egy bájos kis házat Budafok szép panorámás részén, a Rózsavölgy feletti dombon, egy csendes mellékutcában. Az ingatlan 520 négyzetméteres telken helyezkedik el.

A ház alapterülete 35 négyzetméter, ahol egy légtérben található a nappali, amelyet részben nyitott válaszfallal választottak el a konyha-étkező résztől. A fürdőszoba zuhanyzóval és WC-vel egy helyiségben található.

A házba víz, áram és internet is be van kötve. A gáz és csatorna az utcában elérhető, a csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.

A telken található még egy 9 négyzetméteres kis házikó, amely jelenleg szobaként funkcionál.

A kert gondozott és tele van gyümölcsfákkal, így számos lehetőséget kínál: kialakítható rajta több nagy magaságyás, építhető kerti medence vagy finn szauna is.

További információkért és időpont egyeztetésért kérem, hívjon telefonon. Hitellel történő vásárlás esetén bankfüggetlen és díjmentes hiteltanácsadó közreműködését biztosítjuk, aki a teljes folyamaton végigkíséri Önt. Az adásvétel biztonságos és gördülékeny lebonyolítása érdekében tapasztalt ingatlan szakjogászaink szolgáltatását is ajánljuk.